In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 147 (delta 58), reused 136 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 176.08 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb     record_level_embedding.py
featureSelection.py  preprocess.py  transformer.py


In [3]:
%rm -r __pycache__

rm: cannot remove '__pycache__': No such file or directory


In [4]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [6]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [8]:
from featureSelection import BorutaFeatureSelector

selector = BorutaFeatureSelector(max_iter=20)
selected_features, X_top, y = selector.select_features(processed_full_df, target_column='label', top_k_features=50)


2025-07-19 04:37:12.927729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752899833.148004      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752899833.212286      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🧹 Input features shape: (257673, 95), Target shape: (257673,)
🚫 Skipping normalization (already preprocessed)...

🚀 Starting Boruta feature selection...
Iteration: 	1 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	2 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	3 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	4 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	5 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	6 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	7 / 20
Confirmed: 	0
Tentative: 	95
Rejected: 	0
Iteration: 	8 / 20
Confirmed: 	64
Tentative: 	11
Rejected: 	20
Iteration: 	9 / 20
Confirmed: 	64
Tentative: 	11
Rejected: 	20
Iteration: 	10 / 20
Confirmed: 	64
Tentative: 	11
Rejected: 	20
Iteration: 	11 / 20
Confirmed: 	64
Tentative: 	11
Rejected: 	20
Iteration: 	12 / 20
Confirmed: 	66
Tentative: 	9
Rejected: 	20
Iteration: 	13 / 20
Confirmed: 	66
Tentative: 	5
Rejected: 	24
Iteration: 	14 / 20
Confirmed: 	66
Tent

In [10]:
columns_to_keep = selected_features

In [11]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 50)

In [12]:
df_selected.dtypes

dur                  float64
spkts                float64
dpkts                float64
sbytes               float64
dbytes               float64
rate                 float64
sttl                 float64
dttl                 float64
sload                float64
dload                float64
sloss                float64
dloss                float64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                 float64
stcpb                float64
dtcpb                float64
dwin                 float64
tcprtt               float64
synack               float64
ackdat               float64
smean                float64
dmean                float64
trans_depth          float64
response_body_len    float64
ct_srv_src           float64
ct_state_ttl         float64
ct_dst_ltm           float64
ct_src_dport_ltm     float64
ct_dst_sport_ltm     float64
ct_dst_src_ltm       float64
is_ftp_login         float64
ct_ftp_cmd    

In [13]:
numerical_columns = [col for col in columns_to_keep 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_"))]


In [14]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 1 categorical blocks:
   - proto: 11 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: dur
   - Adding numerical input: spkts
   - Adding numerical input: dpkts
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: sinpkt
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: djit
   - Adding numerical input: swin
   - Adding numerical input: stcpb
   - Adding numerical input: dtcpb
   - Adding numerical input: dwin
   - Adding numerical input: tcprtt
   - Adding numerical input:

I0000 00:00:1752900822.086477      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752900822.087075      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


[INFO] Projection complete. Output embedding shape: (64,)
[INFO] Model build complete.
[INFO] Preparing inputs for embedding. Number of rows: 257673
   - Prepared numerical input: dur
   - Prepared numerical input: spkts
   - Prepared numerical input: dpkts
   - Prepared numerical input: sbytes
   - Prepared numerical input: dbytes
   - Prepared numerical input: rate
   - Prepared numerical input: sttl
   - Prepared numerical input: dttl
   - Prepared numerical input: sload
   - Prepared numerical input: dload
   - Prepared numerical input: sloss
   - Prepared numerical input: dloss
   - Prepared numerical input: sinpkt
   - Prepared numerical input: dinpkt
   - Prepared numerical input: sjit
   - Prepared numerical input: djit
   - Prepared numerical input: swin
   - Prepared numerical input: stcpb
   - Prepared numerical input: dtcpb
   - Prepared numerical input: dwin
   - Prepared numerical input: tcprtt
   - Prepared numerical input: synack
   - Prepared numerical input: ackdat
  

I0000 00:00:1752900824.357383     387 service.cc:148] XLA service 0x7b55a4009910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752900824.358222     387 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752900824.358245     387 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752900824.404495     387 cuda_dnn.cc:529] Loaded cuDNN version 90300


  53/8053 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step    

I0000 00:00:1752900824.760055     387 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8053/8053 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step
[INFO] Embedding complete. Output shape: (257673, 64)
[INFO] Embedding DataFrame ready.


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,0.135214,0.345523,0.037953,-0.230606,0.196916,0.609050,-0.163039,-0.597626,0.176523,-0.629823,...,-0.402695,-0.282372,-0.437640,0.542146,-0.122819,0.522201,-0.164128,-0.280107,0.029785,0.202094
1,0.111003,0.339695,0.085009,-0.307963,0.276295,0.539551,-0.331477,-0.270183,0.131982,-0.656045,...,-0.608247,-0.161399,-0.540318,0.324129,-0.175015,0.467181,-0.218719,-0.347534,0.113040,0.118312
2,0.248893,0.404933,0.086955,-0.285031,0.249464,0.546768,-0.293510,-0.420050,0.232435,-0.628394,...,-0.580795,-0.117076,-0.363125,0.314784,-0.151105,0.499729,-0.154956,-0.418193,0.069099,0.141559
3,0.194529,0.225709,0.282912,-0.197300,0.253249,0.634476,-0.282203,-0.527245,0.357347,-0.637771,...,-0.331807,-0.169151,-0.260792,0.444550,-0.036876,0.687566,-0.021668,-0.437405,0.083686,0.058034
4,0.152496,0.397563,0.119520,-0.556581,0.360499,0.623487,-0.385668,-0.442815,0.117849,-0.449727,...,-0.478852,-0.315654,-0.726959,0.005641,-0.420929,0.447488,-0.204079,-0.412522,0.267820,0.063991


In [15]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split
import numpy as np  # Just in case

# Input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Transformer block
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)

x = transformer.apply(input_layer, training=True)

# Classification head
classification_head = LastTokenClassificationHead()
x = classification_head.apply(x)

# MLP head
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

# Output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# Prepare data
X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)

X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Define early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)


[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9000 - loss: 0.2075[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 48s 5ms/step - accuracy: 0.9000 - loss: 0.2075 - val_accuracy: 0.9163 - val_loss: 0.1706
Epoch 2/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - accuracy: 0.9155 - loss: 0.1709 - val_accuracy: 0.9157 - val_loss: 0.1599
Epoch 3/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - accuracy: 0.9202 - loss: 0.1621 - val_accuracy: 0.9192 - val_loss: 0.1579
Epoch 4/50
6442/6442 

In [17]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9336 - loss: 0.1309
Validation loss: 0.1304, Validation accuracy: 0.9344


In [18]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1587/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9488
Confusion Matrix:
 [[16857  1743]
 [ 1636 31299]]
Detection Rate (Recall): 0.9503
False Alarm Rate: 0.0937
